작성일자: 2022-08-01<br>
작성자: 김동연

클래스의 다형성을 이해하기 위해서 MapReduce 구현을 살펴보자.<br>
입력 데이터를 표현할 공통 클래스가 필요하다고 가정해보자.

In [1]:
class InputData(object):
    def read(self):
        raise NotImplementedError

다음은 파일에서 데이터를 읽어오는 InputData의 서브클래스다.

In [2]:
class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
    
    def read(self):
        return open(self.path).read()

이런 서브클래스는 여러 종류가 있을 수 있고 각 서브 클래스는 데이터를 반환하는 표준 인터페이스 read를 구현한다.<br>
입력 데이터를 처리하는 MapReduce 작업 클래스도 비슷한 추상 인터페이스를 작성한다.

In [3]:
class Worker(object):
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
    
    def map(self):
        raise NotImplementedError
    
    def reduce(self, other):
        raise NotImplementedError

줄 바뀐 횟수를 세는 서브클래스

In [4]:
class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
        
    def reduce(self, other):
        self.result += other.result

헬퍼 함수로 객체를 생성하여 각각의 서브클래스들을 연결

In [5]:
def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))
        
def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers

def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads: thread.start()
    for thread in threads: thread.join()
        
    first, rest = workers[0], workers[1:]
    for worker in rest:
        first.reduce(worker)
    return first.result

def mapreduce(data_dir):
    inputs = genreate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

하지만 위의 방법은 전혀 범용적이지 않은 mapreduce 함수를 생성한다.<br>
다른 InputData나 Worker 서브클래스에는 새로운 generate_inputs, create_workers, mapreduce 함수들을 다시 작성해야 한다.<br>
<br>
파이썬은 단일 생성자 매서드 ```__init__```만을 허용하기 때문에 모든 InputData 서브클래스가 호환되는 생성자를 갖출수는 없다.<br>
@classmethod 다형성을 이용하면 문제를 해결할 수 있다.<br>
<br>
새로운 범용 클래스를 작성한다. generate_inputs 매서드로 다형성을 가질 수 있다.

In [6]:
class GenericInputData(object):
    def read(self):
        raise NotImplementedError
    
    
    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

In [7]:
class PathInputData(GenericInputData):
    # ...
    def read(self):
        return open(self.path).read()
    
    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))

마찬가지로 create_workers 메서드로 다형성을 가질 수 있다. 또한 cls(input_data)를 통해 구현한 서브클래스의 인스턴스를 생성한다.<br>
또한 input_class.generate_inputs의 호출로 더 범용적인 구현이 가능하다. 이 호출이 바로 클래스 다형성이다.

In [8]:
class GenericWorker(object):
    #...
    def map(self):
        raise NotImplementedError
    
    def reduce(self, other):
        raise NotImplementedError
    
    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers

In [9]:
class LineCountWorker(GenericWorker):
    #...
    def reduce(self, other):
        raise NotImplementedError

위의 클래스를 사용하여 더 범용적인 mapreduce 함수를 작성할 수 있다.

In [ ]:
def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

with TemporaryDirectory() as tmpdir:
    write_test_fils(tmpdir)
    config = {'data_dir' : tmpdir}
    result = mapreduce(LineCountWorker, PathInputData, config)